# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df.info())
display(df.head(40))
df['children'] = df['children'].replace(-1, 1) # Оптимизация значений в столбце Дети

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Выводы по данным:
1) в столбце total_income (доход) есть пропуски данных, которые не дадут корректно обработать информацию по заемщикам

2) в столбце education (образование) встречается разностильное написание, что для исследования необходимо будет привести к однообразному виду (необходимому для группировок)

3) в столбце days_employed (стаж) данные указаны в отрицательном виде (из-за "-" перед значением), также присутствуют артефакты в виде невозможных 4-х значных значений. Также есть пропуски данных, которые необходимо будет анализировать, на основании значений других столбцов.

4) столбец purpose необходимо будет группировать с помощью лемметизации

5) при изучении данных в столбце дети было выявлено отрицательное значение, которое заменили на положительное, предположив, что это была опечатка. Значение 20 могло быть как ошибкой ввода, так и многодетной семьёй, поэтому это значение решено оставить без изменений.



## Предобработка данных

### Обработка пропусков

In [2]:
display(df[df.isna().any(axis=1)].head(3)) #на начальном этапе - поиск пустых значений для анализа
df['total_income'] = df['total_income'].abs()#подготовили данные в столбце Доход для обработки
df['days_employed'] = df['days_employed'].abs()#подготовили данные в столбце Стаж для обработки

df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

display(df.groupby('income_type').total_income.fillna('median').head(30))

df.head(30) #вывод результатов заполнения пуcтых значений

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости


0     253875.639453
1     112080.014102
2     145885.952297
3     267628.550329
4      158616.07787
5     255763.565419
6      240525.97192
7     135823.934197
8      95856.832424
9     144425.938277
10     113943.49146
11     77069.234271
12           median
13    130458.228857
14    165127.911772
15     116820.90445
16    289202.704229
17     90410.586745
18     56823.777243
19    242831.107982
20    209552.644578
21    131812.203909
22    149681.140731
23    128265.720871
24    290547.235997
25     55112.757732
26           median
27    308848.983691
28    187863.237306
29           median
Name: total_income, dtype: object

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

На этапе изучения информации о данных, были выявлены (на основании сводной информации) пропуски значений в столбцах  days_employed (стаж) и total_income (доход). 

Более того, все пропуски данных были у одних и тех же респондентов (в строке с отсутствием стажа отсутствовала информация о доходе), при этом корреляции между другими данными по этим респондентам не прослеживается. 

Также выявлено, что попуски данных приблизительно у 10% респондентов.

По причинам появления пропусков можно предположить, что данные были изначально намеренно не собраны (возможно в одном из вариантов анкеты не было таких пунктов, а в момент заполнения базы данных не вносились из-за их отсутствия).

Т.к. пропущенные значения являются количественными, то, чтобы наименее исказить исследование, возможно заполнить их медианными значениями соответствующих столбцов таблицы.

### Замена типа данных

In [3]:
df['total_income'] = df['total_income'].fillna(0).astype('int')
display(df.head())
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

В столбце заработок встречаются как положительные, так и отрицательные значения. Также указаны данные типа Float. 
Для приведения в читабельный вид и для корректировки значений, которые были внесены с отрицательным значением (возможно по причине некорректного переноса данных из другой базы), было решено использовать функции:
 - abs() для корректировки отрицательных значений;
 - int() для изменения типа столбца и отброса незначащих данных поле запятой.


### Обработка дубликатов

In [4]:
#df.duplicated().sum() #на начальном этапе - поиск полных дубликатов строк (было выявлено 54 дубликата)
df = df.drop_duplicates()

**Вывод**

Определили полные дубликаты в данных.

Их появление, скорее всего, связано с формированием из нескольких источников данных, в которых были общие источники (респонденты)

### Лемматизация

In [5]:
#Функция для создания идентификатора целей получения кредита
from pymystem3 import Mystem
m = Mystem()

def get_purpose_id(purpose):
    lemma = m.lemmatize(purpose)
    if ('жилье' in lemma) or ('недвижимость' in lemma):
        return 1
    elif 'автомобиль' in lemma:
        return 2
    elif 'образование' in lemma:
        return 3
    else:
        return 4

df['purpose_id'] = df['purpose'].apply(get_purpose_id)
    

df.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4


**Вывод**

Для начала ознакомимся со списком уникальных значений столбца purpose (цель получения кредита)

Из данных можем выделить 4 основных направления: 
-жильё(недвижимость);
-автомобиль;
-образование;
-свадьба.

Т.к. по основным направлениям есть важные уточнения (к примеру приобретение коммерческой или жилой недвижимости), для идентификации цели кредита, лучше создать столбец purpose_id  и присвоить основным направлениям уникальные номера (от 1 до 4).

### Категоризация данных

In [6]:
children_debt=df[['children','debt']]
family_status_debt=df[['family_status_id','debt']]
total_income_debt=df[['total_income','debt']]
purpose_debt=df[['purpose_id','debt']]

**Вывод**

Для ответа на вопросы проекта необходимо будет категоризировать данные по следующим группам:
- наличие детей и наличие задолженности (children/ debt)
- семейное положение и наличие задолженности (family_status_id/ debt), (в котором 0-"женат / замужем", 1-"гражданский брак", 
2-"вдовец / вдова", 3-"в разводе" и "свыдьба") 
- уровень дохода и наличие задолженности (total_income/ debt)
- цель кредита и наличие задолженности (purpose_id/ debt) (в котором "жилье и недвижимость" = 1, "автомобиль" = 2, 
"образование" = 3 и "свадьба" = 4)

Можно предположить, что в столбце debt использовано значение 0 как отсутствие задолженности, а 1 как её наличие.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
def get_children_id(children):
    if children==0:
        return 0
    else:
        return 1 
    
children_debt['children_id']=children_debt['children'].apply(get_children_id)

print('Доля проблемных заемщиков c детьми(1) и без детей(0):', (children_debt.groupby('children_id').debt.mean())*100)

Доля проблемных заемщиков c детьми(1) и без детей(0): children_id
0    7.535266
1    9.206953
Name: debt, dtype: float64


/tmp/ipykernel_93/334328646.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children_debt['children_id']=children_debt['children'].apply(get_children_id)


In [8]:
df.groupby('children').debt.mean()

children
0     0.075353
1     0.091639
2     0.094542
3     0.081818
4     0.097561
5     0.000000
20    0.105263
Name: debt, dtype: float64

**Вывод**

По данным, полученным по результатам исследования выборки о зависимости между наличием детей и возвратом кредита в срок, можно увидеть следующий результат:
 - доля проблемных заемщиков без детей 7,5% против 9,2% с детьми.

Данные показатели можно интерпритировать как вероятность невозврата кредита выше в семьях, где есть дети, но в целом по полученным данным можно допустить, что расхождения в процентном соотношении можно отнести к погрешности выборки.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
# Количество заёмщиков в браке без задолженности
family_status_debt_00 = family_status_debt[(family_status_debt['family_status_id'] == 0)&(family_status_debt['debt'] == 0)]['family_status_id'].count()
# Количество заёмщиков в браке и с задолженностью
family_status_debt_01 = family_status_debt[(family_status_debt['family_status_id'] == 0)&(family_status_debt['debt'] == 1)]['family_status_id'].count()
# Количество заёмщиков в гражданском браке без задолженности
family_status_debt_10 = family_status_debt[(family_status_debt['family_status_id'] == 1)&(family_status_debt['debt'] == 0)]['family_status_id'].count()
# Количество заёмщиков в гражданском браке и с задолженностью
family_status_debt_11 = family_status_debt[(family_status_debt['family_status_id'] == 1)&(family_status_debt['debt'] == 1)]['family_status_id'].count()
# Количество заёмщиков вдов/вдовцов без задолженности
family_status_debt_20 = family_status_debt[(family_status_debt['family_status_id'] == 2)&(family_status_debt['debt'] == 0)]['family_status_id'].count()
# Количество заёмщиков вдов/вдовцов с задолженностью
family_status_debt_21 = family_status_debt[(family_status_debt['family_status_id'] == 2)&(family_status_debt['debt'] == 1)]['family_status_id'].count()
# Количество заёмщиков в разводе без задолженности
family_status_debt_30 = family_status_debt[(family_status_debt['family_status_id'] == 3)&(family_status_debt['debt'] == 0)]['family_status_id'].count()
# Количество заёмщиков в разводе с задолженностью
family_status_debt_31 = family_status_debt[(family_status_debt['family_status_id'] == 3)&(family_status_debt['debt'] == 1)]['family_status_id'].count()
# Количество заёмщиков в разводе без задолженности
family_status_debt_40 = family_status_debt[(family_status_debt['family_status_id'] == 4)&(family_status_debt['debt'] == 0)]['family_status_id'].count()
# Количество заёмщиков в разводе с задолженностью
family_status_debt_41 = family_status_debt[(family_status_debt['family_status_id'] == 4)&(family_status_debt['debt'] == 1)]['family_status_id'].count()

df.groupby('family_status').agg({"debt": ['count', 'sum', 'mean']})


debt               
                       count  sum      mean
family_status                              
Не женат / не замужем   2810  274  0.097509
в разводе               1195   85  0.071130
вдовец / вдова           959   63  0.065693
гражданский брак        4163  388  0.093202
женат / замужем        12344  931  0.075421

**Вывод**

По группе в браке средняя доля проблемных кредитов составляет 7,5%

По группе в гражданском браке средняя доля проблемных кредитов составляет 9,3%

По группе вдов/вдовцов средняя доля проблемных кредитов составляет 6,6%

По группе в разводе средняя доля проблемных кредитов составляет 9%

По группе молодоженов средняя доля проблемных кредитов составляет 10,5%

Из чего можно сделать вывод о том, что люди в браке или в разводе более ответственно относятся к своим обязательствам, чем люди с более свободной жизненной позицией (разведенные, гражданский брак, молодожены)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
# Для начала анализа отсортируем таблицу по возрастанию доходов
total_income_debt=total_income_debt.sort_values(by='total_income')

#На начальном этапе разделение доходов на 3 группы
total_income_debt['quantile_ex_1'] = pd.qcut(total_income_debt['total_income'], q=3)
print(total_income_debt['quantile_ex_1'].unique())

# Разделим данные на основные 3 группы
t_i_d_low_0 = total_income_debt[(total_income_debt['total_income'] < 119253.667)&(total_income_debt['debt'] == 0)]['total_income'].count()
t_i_d_low_1 = total_income_debt[(total_income_debt['total_income'] < 119253.667)&(total_income_debt['debt'] == 1)]['total_income'].count()
t_i_d_med_0 = total_income_debt[(total_income_debt['total_income'] >= 119253.667)&(total_income_debt['total_income'] <= 172357.0)&(total_income_debt['debt'] == 0)]['total_income'].count()
t_i_d_med_1 = total_income_debt[(total_income_debt['total_income'] >= 119253.667)&(total_income_debt['total_income'] <= 172357.0)&(total_income_debt['debt'] == 1)]['total_income'].count()
t_i_d_high_0 = total_income_debt[(total_income_debt['total_income'] > 172357.0)&(total_income_debt['debt'] == 0)]['total_income'].count()
t_i_d_high_1 = total_income_debt[(total_income_debt['total_income'] > 172357.0)&(total_income_debt['debt'] == 1)]['total_income'].count()


print('Процент проблемных кредитов у заёмщиков с низким доходом:',(int((t_i_d_low_1/(t_i_d_low_1+t_i_d_low_0))*10000))/100,'%')
print('Процент проблемных кредитов у заёмщиков со средним доходом:',(int((t_i_d_med_1/(t_i_d_med_1+t_i_d_med_0))*10000))/100,'%')
print('Процент проблемных кредитов у заёмщиков с низким доходом:',(int((t_i_d_high_1/(t_i_d_high_1+t_i_d_high_0))*10000))/100,'%')

[(-0.001, 104698.333], (104698.333, 170764.333], (170764.333, 2265604.0]]
Categories (3, interval[float64]): [(-0.001, 104698.333] < (104698.333, 170764.333] < (170764.333, 2265604.0]]
Процент проблемных кредитов у заёмщиков с низким доходом: 8.06 %
Процент проблемных кредитов у заёмщиков со средним доходом: 8.91 %
Процент проблемных кредитов у заёмщиков с низким доходом: 7.52 %


In [11]:
#код ревьюера
total_income_debt.groupby('quantile_ex_1').debt.mean()

quantile_ex_1
(-0.001, 104698.333]        0.079083
(104698.333, 170764.333]    0.088445
(170764.333, 2265604.0]     0.075730
Name: debt, dtype: float64

**Вывод**

По значениям в поле total_income, можем предположить, что значение похоже на годовой доход.
Исходя из этого, можно разделить данные на основные 3 группы: 
 - низкий доход (до 119253.67),
 - средний (до 172357.0),
 - выше среднего (от 172357.0)

По группе заёмщиков с низким доходом доля проблемных кредитов составляет 8,11%

По группе заёмщиков со средним доходом доля проблемных кредитов составляет 8.66%

По группе заёмщиков с высоким доходом доля проблемных кредитов составляет 7,52%

Исходя из полученных данных, можно сделать вывод о том, что займы для людей с большими доходами менее рискованные.
 

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
# Количество займов на недвижимость без задолженности
purpose_debt_10 = purpose_debt[(purpose_debt['purpose_id'] == 1)&(purpose_debt['debt'] == 0)]['purpose_id'].count()
# Количество займов на недвижимость с задолженностью
purpose_debt_11 = purpose_debt[(purpose_debt['purpose_id'] == 1)&(purpose_debt['debt'] == 1)]['purpose_id'].count()
# Количество займов на автомобиль без задолженности
purpose_debt_20 = purpose_debt[(purpose_debt['purpose_id'] == 2)&(purpose_debt['debt'] == 0)]['purpose_id'].count()
# Количество займов на автомобиль с задолженностью
purpose_debt_21 = purpose_debt[(purpose_debt['purpose_id'] == 2)&(purpose_debt['debt'] == 1)]['purpose_id'].count()
# Количество займов на образование без задолженности
purpose_debt_30 = purpose_debt[(purpose_debt['purpose_id'] == 3)&(purpose_debt['debt'] == 0)]['purpose_id'].count()
# Количество займов на образование с задолженностью
purpose_debt_31 = purpose_debt[(purpose_debt['purpose_id'] == 3)&(purpose_debt['debt'] == 1)]['purpose_id'].count()
# Количество займов на бракосочетание без задолженности
purpose_debt_40 = purpose_debt[(purpose_debt['purpose_id'] == 4)&(purpose_debt['debt'] == 0)]['purpose_id'].count()
# Количество займов на бракосочетание с задолженностью
purpose_debt_41 = purpose_debt[(purpose_debt['purpose_id'] == 4)&(purpose_debt['debt'] == 1)]['purpose_id'].count()

print('Количество займов на недвижимость без задолженности', purpose_debt_10)
print('Количество займов на недвижимость и с задолженностью', purpose_debt_11)
print('Количество займов на автомобиль без задолженности', purpose_debt_20)
print('Количество займов на автомобиль с задолженностью', purpose_debt_21)
print('Количество займов на образование без задолженности', purpose_debt_30)
print('Количество займов на образование с задолженностью', purpose_debt_31)
print('Количество займов на бракосочетание без задолженности', purpose_debt_40)
print('Количество займов на бракосочетание с задолженностью', purpose_debt_41)


Количество займов на недвижимость без задолженности 10032
Количество займов на недвижимость и с задолженностью 782
Количество займов на автомобиль без задолженности 3905
Количество займов на автомобиль с задолженностью 403
Количество займов на образование без задолженности 3644
Количество займов на образование с задолженностью 370
Количество займов на бракосочетание без задолженности 2149
Количество займов на бракосочетание с задолженностью 186


**Вывод**

По группе займов на недвижимость доля проблемных кредитов составляет 7,2%

По группе займов на автомобиль доля проблемных кредитов составляет 9,3%

По группе займов на образование доля проблемных кредитов составляет 9,2%

По группе займов на бракосочетание доля проблемных кредитов составляет 8%

Из чего можно сделать вывод о том, что более успешными займами можно считать займы на недвижимость и бракосочетание. 
Займы же на автомобиль и образование подвержены большим рискам.


In [13]:
#код ревьюера
df.groupby('purpose_id').debt.mean()

purpose_id
1    0.072314
2    0.093547
3    0.092177
4    0.079657
Name: debt, dtype: float64

## Общий вывод

По данной выборке можно сделать следующие выводы:
 - для минимизации издержек по проблемным кредитам, необходимо повышать ставки заёмщикам, которые попадают в группы риска и, напротив, понижать ставки менее рискованным группам – тем самым управляя риском;
 - обработка данных в приведенной выборке была со значительным количеством условных допусков, которые было бы полезно уточнить у предоставляющей информацию стороны до начала анализа (на этапе первичной обработки данных).